In [124]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from yfinance_loader import download_tickers

%matplotlib inline

In [125]:
tickers = pd.read_csv('ticker_lists/ndx.csv', header=None)
tickers = tickers[0].to_list()

In [148]:
tickers = ['BAVA.CO']
stocks = download_tickers(tickers, False, return_only=['Adj Close','Volume'])

In [149]:
print(stocks)

Stock ticker     BAVA.CO        
Data type      Adj Close  Volume
Date                            
2000-01-03     65.972298   22397
2000-01-04     64.102097    3063
2000-01-05     64.690201    2525
2000-01-06     68.610802    1737
2000-01-07     76.452103   10509
...                  ...     ...
2020-09-02    210.800003  273924
2020-09-03    201.800003  420723
2020-09-04    195.000000  858496
2020-09-07    205.600006  475036
2020-09-08    195.500000  361782

[5231 rows x 2 columns]
